In [4]:
import os
import numpy as np
from models.cnn_models import cnn_baseline, cnn_multihead, cnn_cnnhpss
from keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utility.file_utility import FileUtility
from data_utility.labeling_utility import LabelingData
import itertools
from data_utility.feedgenerator import train_batch_generator_408, validation_batch_generator_408
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [5]:
def run_baseline(epochs=10, setting_name='basemodel', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200,use_CRF=False,filter_size=256):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'
    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = cnn_cnnhpss(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, use_CRF=use_CRF, filter_size=filter_size)

    # output directory
    FileUtility.ensure_dir('results_cnn/' + setting_name + params + '/')

    # save model
    with open('results_cnn/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'results_cnn/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max', baseline=None)
    callbacks_list = [earlystopping]#checkpoint]#, 

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('results_cnn/' + setting_name + params + '/history', h.history)
    return model



In [6]:
model=run_baseline(epochs=1000, setting_name='cnn_cnnhpss_', gpu='1', train_batch_size=50,
                 test_batch_size=50, patience=50, features_to_use=['onehot', 'sequence_profile'], convs=[7,9,11], dense_size=1000, use_CRF=True, filter_size=80)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
Tensor("crf1/cond/Merge:0", shape=(?, ?, 9), dtype=float32)
(None, None, 9)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_2[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_5 (Concatenate

Epoch 3/1000
111/111 [==============================] - 32s 292ms/step - loss: 147.2050 - weighted_acc: 0.7023 - val_loss: 264.6507 - val_weighted_acc: 0.6620
Epoch 4/1000
111/111 [==============================] - 33s 294ms/step - loss: 135.8951 - weighted_acc: 0.7111 - val_loss: 254.4420 - val_weighted_acc: 0.6607
Epoch 5/1000
111/111 [==============================] - 33s 301ms/step - loss: 126.9463 - weighted_acc: 0.7172 - val_loss: 246.3191 - val_weighted_acc: 0.6590
Epoch 6/1000
111/111 [==============================] - 34s 305ms/step - loss: 119.5060 - weighted_acc: 0.7225 - val_loss: 240.6140 - val_weighted_acc: 0.6586
Epoch 7/1000
111/111 [==============================] - 34s 307ms/step - loss: 113.8536 - weighted_acc: 0.7250 - val_loss: 236.0492 - val_weighted_acc: 0.6552
Epoch 8/1000
111/111 [==============================] - 34s 304ms/step - loss: 108.2496 - weighted_acc: 0.7303 - val_loss: 235.0536 - val_weighted_acc: 0.6409
Epoch 9/1000
111/111 [========================